In [16]:
import csv
import os
import glob

import numpy as np

In [17]:
behavioral_data_glob = os.path.join("results", "full", "behavioral_data", "*.csv")
files = glob.glob(behavioral_data_glob)
files.sort(key=os.path.getctime)
most_recent_file = files[-1]
most_recent_file

'results/full/behavioral_data/beh__Mezczyzna__-_2022-02-09 16-12.csv'

In [18]:
with open(most_recent_file, "r") as file:
    reader = csv.DictReader(file)
    rows = [row for row in reader]

# cluster into blocks
blocks = []
for previous_row, current_row in zip([dict()] + rows[:-1], rows):
    if previous_row.get("block_type") != current_row.get("block_type"):
        # next_row is in a new block
        blocks.append((current_row.get("block_type"), []))
    # append row to the latest block
    blocks[-1][1].append(current_row)



In [19]:
experiment_block = blocks[-1][1]    # TODO make it general

congruent_correct_rts = []
incongruent_correct_rts = []
congruent_error_rts = []
incongruent_error_rts = []

for row in experiment_block:
    rt = row["rt"]
    if rt == "":
        # no reaction was given
        continue
    rt = float(rt)
    if row["target_name"] in ["congruent_lll", "congruent_rrr"]:
        if row["reaction"] == "correct":
            congruent_correct_rts.append(rt)
        elif row["reaction"] == "incorrect":
            congruent_error_rts.append(rt)
        else:
            raise Exception()
    elif row["target_name"] in ["incongruent_lrl", "incongruent_rlr"]:
        if row["reaction"] == "correct":
            incongruent_correct_rts.append(rt)
        elif row["reaction"] == "incorrect":
            incongruent_error_rts.append(rt)
        else:
            raise Exception()
    else:
        raise Exception()



In [21]:
def stats(data):
    if len(data) < 2:
        return "      -      "
    mean = np.mean(data)
    # use ddof=1 to calculate sample std, not population std
    standard_error = np.std(data, ddof=1) / np.sqrt(len(data))
    return f"{mean:.3f} ± {standard_error:.3f}"


def print_len(data):
    return f"{len(data):8d}     "

In [22]:
print(f"""
REACTION TIMES:
             |     CORRECT     |      ERROR      |       ALL       |
CONGRUENT    |  {stats(congruent_correct_rts)}  |  {stats(congruent_error_rts)}  |  {stats(congruent_correct_rts + congruent_error_rts)}  |
INCONGRUENT  |  {stats(incongruent_correct_rts)}  |  {stats(incongruent_error_rts)}  |  {stats(incongruent_correct_rts + incongruent_error_rts)}  |
ALL          |  {stats(congruent_correct_rts + incongruent_correct_rts)}  |  {stats(congruent_error_rts + incongruent_error_rts)}  |  {stats(congruent_correct_rts + congruent_error_rts + incongruent_correct_rts + incongruent_error_rts)}  |


NUMBER OF TRIALS:
             |     CORRECT     |      ERROR      |       ALL       |
CONGRUENT    |  {print_len(congruent_correct_rts)}  |  {print_len(congruent_error_rts)}  |  {print_len(congruent_correct_rts + congruent_error_rts)}  |
INCONGRUENT  |  {print_len(incongruent_correct_rts)}  |  {print_len(incongruent_error_rts)}  |  {print_len(incongruent_correct_rts + incongruent_error_rts)}  |
ALL          |  {print_len(congruent_correct_rts + incongruent_correct_rts)}  |  {print_len(congruent_error_rts + incongruent_error_rts)}  |  {print_len(congruent_correct_rts + congruent_error_rts + incongruent_correct_rts + incongruent_error_rts)}  |
""")


REACTION TIMES:
             |     CORRECT     |      ERROR      |       ALL       |
CONGRUENT    |  0.278 ± 0.007  |  0.297 ± 0.066  |  0.281 ± 0.011  |
INCONGRUENT  |  0.356 ± 0.017  |  0.249 ± 0.030  |  0.294 ± 0.024  |
ALL          |  0.286 ± 0.007  |  0.273 ± 0.035  |  0.283 ± 0.010  |


NUMBER OF TRIALS:
             |     CORRECT     |      ERROR      |       ALL       |
CONGRUENT    |        41       |         7       |        48       |
INCONGRUENT  |         5       |         7       |        12       |
ALL          |        46       |        14       |        60       |



In [27]:
from collections import Counter

c = Counter()

for trial in experiment_block:
    rt = trial["rt"]
    feedback_type = trial["feedback_type"]
    threshold_rt = trial["threshold_rt"]
    print(rt, feedback_type, threshold_rt)
    c.update([feedback_type])



0.2609570026397705 feedback_good 0.42
0.2914559841156006 feedback_good 0.3086699018478394
0.3106510639190674 feedback_bad 0.27925639152526854
0.2776060104370117 feedback_good 0.2884060859680176
0.3069448471069336 feedback_bad 0.2803760051727295
0.27503204345703125 feedback_good 0.28453099727630615
0.30921006202697754  0.28453099727630615
0.2578001022338867 feedback_good 0.27709121704101564
0.29233789443969727  0.27709121704101564
0.22460222244262695 feedback_good 0.2752894401550293
0.30844807624816895 feedback_bad 0.2665870189666748
0.29182004928588867 feedback_bad 0.270809531211853
0.3581061363220215 feedback_bad 0.27503204345703125
0.22499513626098633 feedback_good 0.2758042335510254
0.2753109931945801  0.2758042335510254
0.31115078926086426  0.2758042335510254
0.308121919631958 feedback_bad 0.26940202713012695
0.3106050491333008 feedback_bad 0.2736245393753052
0.1772761344909668 feedback_good 0.27554683685302733
0.24108386039733887  0.27554683685302733
0.2922990322113037 feedback_ba

In [25]:
c

Counter({'feedback_good': 17, 'feedback_bad': 29, '': 14})